# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Processing...
Done!


Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self): # define the structure
        super().__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=5, padding=2), # conv+bn+relu+pooling
                                    nn.BatchNorm2d(16),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=5, padding=2), # conv+bn+relu+pooling
                                    nn.BatchNorm2d(32),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10) # fully-connected layer
        
    def forward(self, x): # define forward(inference) function(do not need backpopagation function)
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out



    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss() # contain softmax
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad() # zero the stored gradient
        outputs = model(images) # forward
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() # backpopagate loss
        optimizer.step() # update weights        
        
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    cnt = 0
    tbar = tqdm(train_loader) # a progress bar of training process
    for images, labels in tbar:
        outputs = model(images)
        pre_labels = outputs.max(dim=1)[1]
        cnt += (pre_labels == labels).sum()
    
        acc = float(cnt) / len(train_loader) / BATCH_SIZE
        tbar.set_description('Epoch [%d/%d], Training Loss: %.4f, Training Accuracy: %.4f%%' % (epoch+1, NUM_EPOCHS, loss.item(), acc*100))
    
    cnt = 0
    tbar = tqdm(test_loader)
    for images, labels in tbar:
        outputs = model(images)
        pre_labels = outputs.max(dim=1)[1]
        cnt += (pre_labels == labels).sum()
    
        acc = float(cnt) / len(test_loader) / BATCH_SIZE
        tbar.set_description('Epoch [%d/%d], Testing Loss: %.4f, Testing Accuracy: %.4f%%' % (epoch+1, NUM_EPOCHS, loss.item(), acc*100))    
    
    
    


100%|████████████████████████████████████████████████████████████| 468/468 [02:38<00:00,  2.95it/s]
Epoch [1/10], Training Loss: 0.0821, Training Accuracy: 98.3357%: 100%|█| 468/468 [01:23<00:00,  5.61it/s]
Epoch [1/10], Testing Loss: 0.0821, Testing Accuracy: 98.2572%: 100%|█| 78/78 [00:15<00:00,  5.19it/s]
100%|████████████████████████████████████████████████████████████| 468/468 [03:08<00:00,  2.49it/s]
Epoch [2/10], Training Loss: 0.0375, Training Accuracy: 98.9683%: 100%|█| 468/468 [01:44<00:00,  4.48it/s]
Epoch [2/10], Testing Loss: 0.0375, Testing Accuracy: 98.8682%: 100%|█| 78/78 [00:17<00:00,  4.55it/s]
100%|████████████████████████████████████████████████████████████| 468/468 [03:45<00:00,  2.08it/s]
Epoch [3/10], Training Loss: 0.0574, Training Accuracy: 99.1854%: 100%|█| 468/468 [02:14<00:00,  3.47it/s]
Epoch [3/10], Testing Loss: 0.0574, Testing Accuracy: 99.0585%: 100%|█| 78/78 [00:18<00:00,  4.26it/s]
100%|████████████████████████████████████████████████████████████| 468

#### Q5:
Please print the training and testing accuracy.